### XGBoost Classification for Gas Burn Category.
<i> <b>Author:</b> Sheena Bagga, Mu Sigma </i> </br>
<i> <b>Date:</b> 04-December-2019 </i> </br>
<i> <b>Version:</b> 1.0 </i>

The purpose of the notebook is to perform XGBoost Regression for predicting the Burn Category of a Unit on the Day level ADS created.

In [2]:
%sh 
/databricks/python/bin/pip install --proxy http://10.245.5.249:8080 RegscorePy
/databricks/python/bin/pip install --proxy http://10.245.5.249:8080 scikit-plot
/databricks/python/bin/pip install --proxy http://10.245.5.249:8080 imblearn
/databricks/python/bin/pip install --proxy http://10.245.5.249:8080 mlflow
/databricks/python/bin/pip install --proxy http://10.245.5.249:8080 xgboost
/databricks/python/bin/pip install --proxy http://10.245.5.249:8080 sklearn
/databricks/python/bin/pip install --proxy http://10.245.5.249:8080 yellowbrick

Requirement already satisfied: RegscorePy in /databricks/python3/lib/python3.6/site-packages (1.1)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.6/site-packages (from RegscorePy) (0.23.0)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.6/site-packages (from RegscorePy) (1.16.2)
Requirement already satisfied: python-dateutil>=2.5.0 in /databricks/python3/lib/python3.6/site-packages (from pandas->RegscorePy) (2.7.3)
Requirement already satisfied: pytz>=2011k in /databricks/python3/lib/python3.6/site-packages (from pandas->RegscorePy) (2018.4)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.6/site-packages (from python-dateutil>=2.5.0->pandas->RegscorePy) (1.11.0)
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already satisfied: scikit-plot in /databricks/python3/lib/python3.6/site-packages (0.3.7)
Requirement already satisfied: scipy>=0.9 in /databricks/python3/lib/python3.6/site-packages (from scikit-plot) (1.1.0)
Requirement already satisfied: matplotlib>=1.4.0 in /databricks/python3/lib/python3.6/site-packages (from scikit-plot) (2.2.2)
Requirement already satisfied: scikit-learn>=0.18 in /databricks/python3/lib/python3.6/site-packages (from scikit-plot) (0.22)
Requirement already satisfied: joblib>=0.10 in /databricks/python3/lib/python3.6/site-packages (from scikit-plot) (0.14.0)
Requirement already satisfied: numpy>=1.7.1 in /databricks/python3/lib/python3.6/site-packages (from matplotlib>=1.4.0->scikit-plot) (1.16.2)
Requirement already satisfied: cycler>=0.10 in /databricks/python3/lib/python3.6/site-packages (from matplotlib>=1.4.0->scikit-plot) (0.10.0)
Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /databricks/python3/lib/python3.6/site-packages (from matplotlib>=1.4.0->scikit-plot) (2.2.0)
Requirement already satisfied: python-dateutil>=2.1 in /databricks/python3/lib/python3.6/site-packages (from matplotlib>=1.4.0->scikit-plot) (2.7.3)
Requirement already satisfied: pytz in /databricks/python3/lib/python3.6/site-packages (from matplotlib>=1.4.0->scikit-plot) (2018.4)
Requirement already satisfied: six>=1.10 in /databricks/python3/lib/python3.6/site-packages (from matplotlib>=1.4.0->scikit-plot) (1.11.0)
Requirement already satisfied: kiwisolver>=1.0.1 in /databricks/python3/lib/python3.6/site-packages (from matplotlib>=1.4.0->scikit-plot) (1.1.0)
Requirement already satisfied: setuptools in /databricks/python3/lib/python3.6/site-packages (from kiwisolver>=1.0.1->matplotlib>=1.4.0->scikit-plot) (39.1.0)
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already satisfied: imblearn in /databricks/python3/lib/python3.6/site-packages (0.0)
Requirement already satisfied: imbalanced-learn in /databricks/python3/lib/python3.6/site-packages (from imblearn) (0.5.0)
Requirement already satisfied: scipy>=0.17 in /databricks/python3/lib/python3.6/site-packages (from imbalanced-learn->imblearn) (1.1.0)
Requirement already satisfied: numpy>=1.11 in /databricks/python3/lib/python3.6/site-packages (from imbalanced-learn->imblearn) (1.16.2)
Requirement already satisfied: scikit-learn>=0.21 in /databricks/python3/lib/python3.6/site-packages (from imbalanced-learn->imblearn) (0.22)
Requirement already satisfied: joblib>=0.11 in /databricks/python3/lib/python3.6/site-packages (from imbalanced-learn->imblearn) (0.14.0)
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already satisfied: mlflow in /databricks/python3/lib/python3.6/site-packages (1.0.0)
Requirement already satisfied: alembic in /databricks/python3/lib/python3.6/site-packages (from mlflow) (1.3.1)
Requirement already satisfied: click>=7.0 in /databri

In [3]:
# Importing the libraries

from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import datetime as dt
import numpy as np
import datetime
import six
import mlflow
import mlflow.sklearn
from RegscorePy import *
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import roc_auc_score
#from imblearn.over_sampling import SMOTE 
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from sklearn.model_selection import cross_val_score
#from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.linear_model import Ridge
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import ResidualsPlot
import pandas as pd
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
# from sklearn.metrics import precision_recall_curve
# from sklearn.metrics import f1_score
# from sklearn.metrics import auc
from sklearn.preprocessing import binarize
import scikitplot as skplt
# from sklearn.multiclass import OneVsRestClassifier
# from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import warnings
warnings.simplefilter(action='ignore')
import statsmodels.api as sm
from scipy import stats

In [4]:
al1 = ["AL1_mmbtu_new_lag1","AL1_mmbtu_new_lag2","Consumer_burbank_temp_actual_lag1","Consumer_burbank_temp_actual_lag2","Consumer_burbank_temp_f","Consumer_burbank_temp_f_lead1","Consumer_burbank_tempF_degreedays","Consumer_burbank_tempF_degreedays_lead1","Consumer_burbank_windspeedKmph_f_lead1","Consumer_long_beach_temp_actual_lag1","Consumer_long_beach_temp_actual_lag2","Consumer_long_beach_temp_f","Consumer_long_beach_temp_f_lead1","Consumer_long_beach_temp_f_lead2","Consumer_long_beach_tempF_degreedays","Consumer_long_beach_tempF_degreedays_lead1","Consumer_long_beach_tempF_degreedays_lead2","Consumer_long_beach_windspeedKmph_f","Consumer_los_angeles_temp_actual_lag1","Consumer_los_angeles_temp_f","Consumer_los_angeles_temp_f_lead1","Consumer_los_angeles_temp_f_lead2","Consumer_los_angeles_tempF_degreedays","Consumer_los_angeles_tempF_degreedays_lead1","Consumer_los_angeles_tempF_degreedays_lead2","Consumer_Temperature_KBUR_actual_degreedays_lag1","Consumer_Temperature_KBUR_actual_degreedays_lag2","Consumer_Temperature_KLAX_actual_degreedays_lag1","Consumer_Temperature_KLGB_actual_degreedays_lag1","Consumer_Temperature_KLGB_actual_degreedays_lag2","DAM","DAM_lead1","DAM_lead2","Gen_santa_margarita_cloudcover_f_lead2","Gen_SP_15_Wind_forecasted_lead1","Gen_tehachapi_windspeedKmph_f","SCE_Load_SP_15_lag1","SCE_Load_SP_15_lag2","Flag_AL1_lag1","Flag_AL1_lag2","AL1_mmbtu_new"]

al2 = ["AL2_mmbtu_new_lag1","AL2_mmbtu_new_lag2","Consumer_burbank_temp_actual_lag1","Consumer_burbank_temp_actual_lag2","Consumer_burbank_temp_f","Consumer_burbank_temp_f_lead1","Consumer_burbank_tempF_degreedays","Consumer_burbank_tempF_degreedays_lead1","Consumer_long_beach_temp_actual_lag1","Consumer_long_beach_temp_actual_lag2","Consumer_long_beach_temp_f","Consumer_long_beach_temp_f_lead1","Consumer_long_beach_tempF_degreedays","Consumer_long_beach_tempF_degreedays_lead1","Consumer_los_angeles_temp_actual_lag1","Consumer_los_angeles_temp_f","Consumer_los_angeles_temp_f_lead1","Consumer_los_angeles_tempF_degreedays","Consumer_los_angeles_tempF_degreedays_lead1","Consumer_Temperature_KBUR_actual_degreedays_lag1","Consumer_Temperature_KBUR_actual_degreedays_lag2","Consumer_Temperature_KLAX_actual_degreedays_lag1","Consumer_Temperature_KLGB_actual_degreedays_lag1","Consumer_Temperature_KLGB_actual_degreedays_lag2","DAM","DAM_lead1","DAM_lead2","Gen_huron_cloudcover_f_lead1","SCE_Load_SP_15_lag1","SCE_Load_SP_15_lag2","Flag_AL2_lag1","Flag_AL2_lag2","AL2_mmbtu_new"]

al3 = ["AL3_mmbtu_new_lag1","AL3_mmbtu_new_lag2","Consumer_burbank_temp_actual_lag1","Consumer_burbank_temp_actual_lag2","Consumer_burbank_temp_f","Consumer_burbank_temp_f_lead1","Consumer_burbank_temp_f_lead2","Consumer_burbank_tempF_degreedays","Consumer_burbank_tempF_degreedays_lead1","Consumer_burbank_tempF_degreedays_lead2","Consumer_long_beach_temp_actual_lag1","Consumer_long_beach_temp_actual_lag2","Consumer_long_beach_temp_f","Consumer_long_beach_temp_f_lead1","Consumer_long_beach_tempF_degreedays","Consumer_long_beach_tempF_degreedays_lead1","Consumer_los_angeles_temp_actual_lag1","Consumer_los_angeles_temp_actual_lag2","Consumer_los_angeles_temp_f","Consumer_los_angeles_temp_f_lead1","Consumer_los_angeles_temp_f_lead2","Consumer_los_angeles_tempF_degreedays","Consumer_los_angeles_tempF_degreedays_lead1","Consumer_los_angeles_tempF_degreedays_lead2","Consumer_Temperature_KBUR_actual_degreedays_lag1","Consumer_Temperature_KBUR_actual_degreedays_lag2","Consumer_Temperature_KLAX_actual_degreedays_lag1","Consumer_Temperature_KLAX_actual_degreedays_lag2","Consumer_Temperature_KLGB_actual_degreedays_lag1","Consumer_Temperature_KLGB_actual_degreedays_lag2","DAM","DAM_lead1","DAM_lead2","SCE_Load_SP_15_lag1","SCE_Load_SP_15_lag2","Flag_AL3_lag1","Flag_AL3_lag2","AL3_mmbtu_new"]

al4 = ["AL4_mmbtu_new_lag1","AL4_mmbtu_new_lag2","Consumer_burbank_dewpoint_f_lead1","Consumer_burbank_dewpoint_f_lead2","Consumer_burbank_temp_actual_lag1","Consumer_burbank_temp_actual_lag2","Consumer_burbank_temp_f","Consumer_burbank_temp_f_lead1","Consumer_burbank_temp_f_lead2","Consumer_burbank_tempF_degreedays","Consumer_burbank_tempF_degreedays_lead1","Consumer_burbank_tempF_degreedays_lead2","Consumer_long_beach_dewpoint_f","Consumer_long_beach_dewpoint_f_lead1","Consumer_long_beach_dewpoint_f_lead2","Consumer_long_beach_temp_actual_lag1","Consumer_long_beach_temp_actual_lag2","Consumer_long_beach_temp_f","Consumer_long_beach_temp_f_lead1","Consumer_long_beach_temp_f_lead2","Consumer_long_beach_tempF_degreedays","Consumer_long_beach_tempF_degreedays_lead1","Consumer_long_beach_tempF_degreedays_lead2","Consumer_los_angeles_dewpoint_f_lead2","Consumer_los_angeles_temp_actual_lag1","Consumer_los_angeles_temp_actual_lag2","Consumer_los_angeles_temp_f","Consumer_los_angeles_temp_f_lead1","Consumer_los_angeles_temp_f_lead2","Consumer_los_angeles_tempF_degreedays","Consumer_los_angeles_tempF_degreedays_lead1","Consumer_los_angeles_tempF_degreedays_lead2","Consumer_Temperature_KBUR_actual_degreedays_lag1","Consumer_Temperature_KBUR_actual_degreedays_lag2","Consumer_Temperature_KLAX_actual_degreedays_lag1","Consumer_Temperature_KLAX_actual_degreedays_lag2","Consumer_Temperature_KLGB_actual_degreedays_lag1","Consumer_Temperature_KLGB_actual_degreedays_lag2","DAM","DAM_lead1","DAM_lead2","SCE_Load_SP_15_lag1","SCE_Load_SP_15_lag2","Flag_AL4_lag1","Flag_AL4_lag2","AL4_mmbtu_new"]

al5 = ["AL5_mmbtu_new_lag1","AL5_mmbtu_new_lag2","Consumer_burbank_temp_actual_lag1","Consumer_burbank_temp_actual_lag2","Consumer_burbank_temp_f","Consumer_burbank_temp_f_lead1","Consumer_burbank_temp_f_lead2","Consumer_burbank_tempF_degreedays","Consumer_burbank_tempF_degreedays_lead1","Consumer_burbank_tempF_degreedays_lead2","Consumer_long_beach_dewpoint_f","Consumer_long_beach_dewpoint_f_lead1","Consumer_long_beach_temp_actual_lag1","Consumer_long_beach_temp_actual_lag2","Consumer_long_beach_temp_f","Consumer_long_beach_temp_f_lead1","Consumer_long_beach_tempF_degreedays","Consumer_long_beach_tempF_degreedays_lead1","Consumer_los_angeles_temp_actual_lag1","Consumer_los_angeles_temp_actual_lag2","Consumer_los_angeles_temp_f","Consumer_los_angeles_temp_f_lead1","Consumer_los_angeles_temp_f_lead2","Consumer_los_angeles_tempF_degreedays","Consumer_los_angeles_tempF_degreedays_lead1","Consumer_los_angeles_tempF_degreedays_lead2","Consumer_Temperature_KBUR_actual_degreedays_lag1","Consumer_Temperature_KBUR_actual_degreedays_lag2","Consumer_Temperature_KLAX_actual_degreedays_lag1","Consumer_Temperature_KLAX_actual_degreedays_lag2","Consumer_Temperature_KLGB_actual_degreedays_lag1","Consumer_Temperature_KLGB_actual_degreedays_lag2","DAM","DAM_lead1","DAM_lead2","Gen_gustine_cloudcover_f_lead2","SCE_Load_SP_15_lag1","SCE_Load_SP_15_lag2","Flag_AL5_lag1","Flag_AL5_lag2","AL5_mmbtu_new"]

al6 = ["AL6_mmbtu_new_lag1","AL6_mmbtu_new_lag2","Consumer_burbank_temp_actual_lag1","Consumer_burbank_temp_actual_lag2","Consumer_burbank_temp_f","Consumer_burbank_temp_f_lead1","Consumer_burbank_temp_f_lead2","Consumer_burbank_tempF_degreedays","Consumer_burbank_tempF_degreedays_lead1","Consumer_burbank_tempF_degreedays_lead2","Consumer_long_beach_dewpoint_f","Consumer_long_beach_dewpoint_f_lead1","Consumer_long_beach_dewpoint_f_lead2","Consumer_long_beach_temp_actual_lag1","Consumer_long_beach_temp_actual_lag2","Consumer_long_beach_temp_f","Consumer_long_beach_temp_f_lead1","Consumer_long_beach_temp_f_lead2","Consumer_long_beach_tempF_degreedays","Consumer_long_beach_tempF_degreedays_lead1","Consumer_long_beach_tempF_degreedays_lead2","Consumer_los_angeles_temp_actual_lag1","Consumer_los_angeles_temp_actual_lag2","Consumer_los_angeles_temp_f","Consumer_los_angeles_temp_f_lead1","Consumer_los_angeles_temp_f_lead2","Consumer_los_angeles_tempF_degreedays","Consumer_los_angeles_tempF_degreedays_lead1","Consumer_los_angeles_tempF_degreedays_lead2","Consumer_Temperature_KBUR_actual_degreedays_lag1","Consumer_Temperature_KBUR_actual_degreedays_lag2","Consumer_Temperature_KLAX_actual_degreedays_lag1","Consumer_Temperature_KLAX_actual_degreedays_lag2","Consumer_Temperature_KLGB_actual_degreedays_lag1","Consumer_Temperature_KLGB_actual_degreedays_lag2","DAM","DAM_lead1","DAM_lead2","Gen_mojave_windspeedKmph_f","Gen_tehachapi_windspeedKmph_f_lead1","SCE_Load_SP_15_lag1","SCE_Load_SP_15_lag2","Flag_AL6_lag1","Flag_AL6_lag2","AL6_mmbtu_new"]

hb1 = [	'Consumer_long_beach_temp_f_lead1','Consumer_los_angeles_temp_actual_lag1',	'Consumer_los_angeles_temp_f_lead1',		'Consumer_los_angeles_tempF_degreedays_lead1',	'DAM',	"DAM_lead2",'HB1_mmbtu_new_lag1','Consumer_Temperature_KBUR_actual_degreedays_lag2',	'SCE_Load_SP_15_lag1','Consumer_burbank_temp_actual_lag2','Consumer_los_angeles_temp_actual_lag2','DAM_lead1','SCE_Load_SP_15_lag2',	'Flag_HB1_lag1','HB1_mmbtu_new']

# hb12 = ['HB1_mmbtu_new_lag1' ,   'HB1_mmbtu_new_lag2' ,'DAM' ,  'DAM_lead1' ,   'SCE_Load_SP_15_lag1' , 'DAM_lead2' , 'Consumer_long_beach_temp_actual_lag1' ,     'Consumer_burbank_temp_actual_lag2',    'Consumer_burbank_tempF_degreedays' ,     'Consumer_long_beach_temp_actual_lag2' , 'Consumer_los_angeles_temp_f_lead1','Consumer_burbank_temp_f_lead2' ,'SCE_Load_SP_15_lag2',   'Flag_HB1_lag1' ,'HB1_mmbtu_new']

   
# hb1 = ['DAM_lead2', 'Consumer_burbank_temp_actual_lag2', 'Consumer_burbank_tempF_degreedays', 'Consumer_los_angeles_temp_actual_lag1', 'Consumer_los_angeles_tempF_degreedays_lead1', 'Consumer_long_beach_temp_f_lead1', 'SCE_Load_SP_15_lag2', 'Consumer_long_beach_temp_actual_lag2', 'Consumer_los_angeles_temp_actual_lag2',  'HB1_mmbtu_new_lag2', 'Consumer_Temperature_KBUR_actual_degreedays_lag2', 'Consumer_burbank_temp_f_lead2', 'HB1_mmbtu_new_lag1', 'Consumer_long_beach_temp_actual_lag1', 'SCE_Load_SP_15_lag1', 'DAM', 'Consumer_los_angeles_temp_f_lead1', 'DAM_lead1','Flag_HB1_lag1', 'HB1_mmbtu_new']


# hb1 = ["Consumer_long_beach_temp_f","Consumer_los_angeles_tempF_degreedays","Consumer_los_angeles_temp_f_lead1","Consumer_Temperature_KLGB_actual_degreedays_lag2", "Consumer_los_angeles_temp_actual_lag2","Consumer_burbank_temp_actual_lag1","Consumer_long_beach_temp_f_lead2","Consumer_Temperature_KBUR_actual_degreedays_lag2", "Consumer_Temperature_KLGB_actual_degreedays_lag1","Consumer_burbank_temp_f_lead1","Consumer_Temperature_KLAX_actual_degreedays_lag1","DAM_lead2", "Consumer_burbank_temp_f_lead2","Consumer_long_beach_tempF_degreedays_lead2","HB1_mmbtu_new_lag2","Consumer_burbank_tempF_degreedays_lead1", "Consumer_Temperature_KBUR_actual_degreedays_lag1","Consumer_long_beach_temp_f_lead1","Consumer_long_beach_temp_actual_lag1","Consumer_long_beach_temp_actual_lag2","Consumer_long_beach_tempF_degreedays","Consumer_los_angeles_temp_f",'HB1_mmbtu_new']

# hb1 = ['Consumer_burbank_temp_actual_lag1',	'Consumer_burbank_temp_actual_lag2',	'Consumer_burbank_temp_f',	'Consumer_burbank_temp_f_lead1',	'Consumer_burbank_temp_f_lead2',	'Consumer_burbank_tempF_degreedays',	'Consumer_burbank_tempF_degreedays_lead1',	'Consumer_burbank_tempF_degreedays_lead2',	'Consumer_long_beach_temp_actual_lag1',	'Consumer_long_beach_temp_actual_lag2',	'Consumer_long_beach_temp_f',	'Consumer_long_beach_temp_f_lead1',	'Consumer_long_beach_temp_f_lead2',	'Consumer_long_beach_tempF_degreedays',	'Consumer_long_beach_tempF_degreedays_lead1',	'Consumer_long_beach_tempF_degreedays_lead2',	'Consumer_los_angeles_temp_actual_lag1',	'Consumer_los_angeles_temp_actual_lag2',	'Consumer_los_angeles_temp_f',	'Consumer_los_angeles_temp_f_lead1',	'Consumer_los_angeles_temp_f_lead2',	'Consumer_los_angeles_tempF_degreedays',	'Consumer_los_angeles_tempF_degreedays_lead1',	'Consumer_los_angeles_tempF_degreedays_lead2',	'Consumer_Temperature_KBUR_actual_degreedays_lag1',	'Consumer_Temperature_KBUR_actual_degreedays_lag2',	'Consumer_Temperature_KLAX_actual_degreedays_lag1',	'Consumer_Temperature_KLAX_actual_degreedays_lag2',	'Consumer_Temperature_KLGB_actual_degreedays_lag1',	'Consumer_Temperature_KLGB_actual_degreedays_lag2',	'DAM',	'DAM_lead1',	'DAM_lead2',	'HB1_mmbtu_new_lag1',	'HB1_mmbtu_new_lag2',	'SCE_Load_SP_15_lag1',	'SCE_Load_SP_15_lag2',	'Flag_HB1_lag1',	'Flag_HB1_lag2','HB1_mmbtu_new']

hb2 = ['Consumer_burbank_dewpoint_f_lead2',	'Consumer_burbank_temp_actual_lag1',	'Consumer_burbank_temp_actual_lag2',	'Consumer_burbank_temp_f',	'Consumer_burbank_temp_f_lead1',	'Consumer_burbank_tempF_degreedays',	'Consumer_burbank_tempF_degreedays_lead1',	'Consumer_burbank_tempF_degreedays_lead2',	'Consumer_long_beach_dewpoint_f_lead1',	'Consumer_long_beach_dewpoint_f_lead2',	'Consumer_long_beach_temp_actual_lag1',	'Consumer_long_beach_temp_actual_lag2',	'Consumer_long_beach_temp_f',	'Consumer_long_beach_temp_f_lead1',	'Consumer_long_beach_tempF_degreedays',	'Consumer_long_beach_tempF_degreedays_lead1',	'Consumer_los_angeles_dewpoint_f_lead2',	'Consumer_los_angeles_temp_actual_lag1',	'Consumer_los_angeles_temp_actual_lag2',	'Consumer_los_angeles_temp_f',	'Consumer_los_angeles_temp_f_lead1',	'Consumer_los_angeles_tempF_degreedays',	'Consumer_los_angeles_tempF_degreedays_lead1',	'Consumer_Temperature_KBUR_actual_degreedays_lag1',	'Consumer_Temperature_KBUR_actual_degreedays_lag2',	'Consumer_Temperature_KLAX_actual_degreedays_lag1',	'Consumer_Temperature_KLAX_actual_degreedays_lag2',	'Consumer_Temperature_KLGB_actual_degreedays_lag1',	'Consumer_Temperature_KLGB_actual_degreedays_lag2',	'DAM',	'DAM_lead1',	'DAM_lead2',	'Gen_mojave_windspeedKmph_f',	'HB2_mmbtu_new_lag1',	'HB2_mmbtu_new_lag2',	'Ratio2_Curtailed_Sce_load',	'SCE_Load_SP_15_lag1',	'SCE_Load_SP_15_lag2',	'Flag_HB2_lag1',	'Flag_HB2_lag2','HB2_mmbtu_new']

rb5 = ['Consumer_burbank_temp_actual_lag1',	'Consumer_burbank_temp_actual_lag2',	'Consumer_burbank_temp_f',	'Consumer_burbank_temp_f_lead1',	'Consumer_burbank_temp_f_lead2',	'Consumer_burbank_tempF_degreedays',	'Consumer_burbank_tempF_degreedays_lead1',	'Consumer_burbank_tempF_degreedays_lead2',	'Consumer_long_beach_dewpoint_f_lead2',	'Consumer_long_beach_temp_actual_lag1',	'Consumer_long_beach_temp_actual_lag2',	'Consumer_long_beach_temp_f',	'Consumer_long_beach_temp_f_lead1',	'Consumer_long_beach_temp_f_lead2',	'Consumer_long_beach_tempF_degreedays',	'Consumer_long_beach_tempF_degreedays_lead1',	'Consumer_long_beach_tempF_degreedays_lead2',	'Consumer_los_angeles_temp_actual_lag1',	'Consumer_los_angeles_temp_f',	'Consumer_los_angeles_temp_f_lead1',	'Consumer_los_angeles_temp_f_lead2',	'Consumer_los_angeles_tempF_degreedays',	'Consumer_los_angeles_tempF_degreedays_lead1',	'Consumer_los_angeles_tempF_degreedays_lead2',	'Consumer_Temperature_KBUR_actual_degreedays_lag1',	'Consumer_Temperature_KBUR_actual_degreedays_lag2',	'Consumer_Temperature_KLAX_actual_degreedays_lag1',	'Consumer_Temperature_KLGB_actual_degreedays_lag1',	'Consumer_Temperature_KLGB_actual_degreedays_lag2',	'DAM',	'DAM_lead1',	'DAM_lead2',	'Gen_mendota_cloudcover_f',	'Gen_mojave_windspeedKmph_f',	'Gen_shandon_cloudcover_f_lead2',	'Gen_tehachapi_windspeedKmph_f',	'Gen_tehachapi_windspeedKmph_f_lead1',	'RB5_mmbtu_new_lag1',	'RB5_mmbtu_new_lag2',	'SCE_Load_SP_15_lag1',	'SCE_Load_SP_15_lag2',		'Flag_RB5_lag1',	'Flag_RB5_lag2','RB5_mmbtu_new']

rb6 = ['Consumer_burbank_temp_actual_lag1',	'Consumer_burbank_temp_actual_lag2',	'Consumer_burbank_temp_f',	'Consumer_burbank_temp_f_lead1',	'Consumer_burbank_temp_f_lead2',	'Consumer_burbank_tempF_degreedays',	'Consumer_burbank_tempF_degreedays_lead1',	'Consumer_burbank_tempF_degreedays_lead2',	'Consumer_long_beach_rh_actual_lag2',	'Consumer_long_beach_temp_actual_lag1',	'Consumer_long_beach_temp_actual_lag2',	'Consumer_long_beach_temp_f',	'Consumer_long_beach_temp_f_lead1',	'Consumer_long_beach_temp_f_lead2',	'Consumer_long_beach_tempF_degreedays',	'Consumer_long_beach_tempF_degreedays_lead1',	'Consumer_long_beach_tempF_degreedays_lead2',	'Consumer_los_angeles_rh_actual_lag2',	'Consumer_los_angeles_temp_actual_lag1',	'Consumer_los_angeles_temp_f',	'Consumer_los_angeles_temp_f_lead1',	'Consumer_los_angeles_temp_f_lead2',	'Consumer_los_angeles_tempF_degreedays',	'Consumer_los_angeles_tempF_degreedays_lead1',	'Consumer_los_angeles_tempF_degreedays_lead2',	'Consumer_Temperature_KBUR_actual_degreedays_lag1',	'Consumer_Temperature_KBUR_actual_degreedays_lag2',	'Consumer_Temperature_KLAX_actual_degreedays_lag1',	'Consumer_Temperature_KLGB_actual_degreedays_lag1',	'Consumer_Temperature_KLGB_actual_degreedays_lag2',	'DAM',	'DAM_lead1',	'DAM_lead2',	'RB6_mmbtu_new_lag1',	'RB6_mmbtu_new_lag2',	'SCE_Load_SP_15_lag1',	'SCE_Load_SP_15_lag2',	'Flag_RB6_lag1',	'Flag_RB6_lag2','RB6_mmbtu_new']

rb7 = ['Consumer_burbank_rh_actual_lag1',	'Consumer_burbank_temp_actual_lag1',	'Consumer_burbank_temp_actual_lag2',	'Consumer_burbank_temp_f',	'Consumer_burbank_temp_f_lead1',	'Consumer_burbank_tempF_degreedays',	'Consumer_burbank_tempF_degreedays_lead1',	'Consumer_burbank_tempF_degreedays_lead2',	'Consumer_long_beach_temp_actual_lag1',	'Consumer_long_beach_temp_actual_lag2',	'Consumer_long_beach_temp_f',	'Consumer_long_beach_temp_f_lead1',	'Consumer_long_beach_tempF_degreedays',	'Consumer_long_beach_tempF_degreedays_lead1',	'Consumer_los_angeles_temp_actual_lag1',	'Consumer_los_angeles_temp_actual_lag2',	'Consumer_los_angeles_temp_f',	'Consumer_los_angeles_temp_f_lead1',	'Consumer_los_angeles_tempF_degreedays',	'Consumer_los_angeles_tempF_degreedays_lead1',	'Consumer_Temperature_KBUR_actual_degreedays_lag1',	'Consumer_Temperature_KBUR_actual_degreedays_lag2',	'Consumer_Temperature_KLAX_actual_degreedays_lag1',	'Consumer_Temperature_KLAX_actual_degreedays_lag2',	'Consumer_Temperature_KLGB_actual_degreedays_lag1',	'Consumer_Temperature_KLGB_actual_degreedays_lag2',	'DAM',	'DAM_lead1',	'DAM_lead2',	'Gen_gustine_cloudcover_f_lead1',	'Gen_gustine_cloudcover_f_lead2',	'Gen_mojave_windspeedKmph_f',	'RB7_mmbtu_new_lag1',	'RB7_mmbtu_new_lag2',	'SCE_Load_SP_15_lag1',	'SCE_Load_SP_15_lag2',	'Flag_RB7_lag1',	'Flag_RB7_lag2','RB7_mmbtu_new']

rb8 = ['Consumer_burbank_temp_actual_lag1',	'Consumer_burbank_temp_actual_lag2',	'Consumer_burbank_temp_f',	'Consumer_burbank_temp_f_lead1',	'Consumer_burbank_temp_f_lead2',	'Consumer_burbank_tempF_degreedays',	'Consumer_burbank_tempF_degreedays_lead1',	'Consumer_burbank_tempF_degreedays_lead2',	'Consumer_burbank_windspeedKmph_f',	'Consumer_long_beach_temp_actual_lag1',	'Consumer_long_beach_temp_actual_lag2',	'Consumer_long_beach_temp_f',	'Consumer_long_beach_temp_f_lead1',	'Consumer_long_beach_tempF_degreedays',	'Consumer_long_beach_tempF_degreedays_lead1',	'Consumer_los_angeles_temp_actual_lag1',	'Consumer_los_angeles_temp_f',	'Consumer_los_angeles_temp_f_lead1',	'Consumer_los_angeles_temp_f_lead2',	'Consumer_los_angeles_tempF_degreedays',	'Consumer_los_angeles_tempF_degreedays_lead1',	'Consumer_los_angeles_tempF_degreedays_lead2',	'Consumer_Temperature_KBUR_actual_degreedays_lag1',	'Consumer_Temperature_KBUR_actual_degreedays_lag2',	'Consumer_Temperature_KLAX_actual_degreedays_lag1',	'Consumer_Temperature_KLGB_actual_degreedays_lag1',	'Consumer_Temperature_KLGB_actual_degreedays_lag2',	'DAM',	'DAM_lead1',	'DAM_lead2',	'RB8_mmbtu_new_lag1',	'RB8_mmbtu_new_lag2',	'SCE_Load_SP_15_lag1',	'SCE_Load_SP_15_lag2',	'Flag_RB8_lag1',	'Flag_RB8_lag2','RB8_mmbtu_new']

In [5]:
#units = [al1,al2,al3,al4,al5,al6,hb1,hb2,rb5,rb6,rb7,rb8]
units = [hb1]

In [6]:
# Pulling the ADS from Blob via an SQL wrapper of PySpark command
adsDay = spark.sql("select * from mu_dispatch_opt.ADS_DayLevel_3December2019_master")
type(adsDay)

# dataset contains the ADS at day level which is used for training
dataset = adsDay.toPandas()
print("dataset_shape : ",dataset.shape)

dataset_shape : (1030, 403)

In [7]:
dataset = dataset[(dataset['Date'] >= '2017-01-01') & (dataset['Date'] <= '2019-06-30')]
print("dataset_shape : ",dataset.shape)

dataset_shape : (909, 403)

In [8]:
# Dictionary to change categorical to numerical
fl = {
      "On":1,
      "Off":0,
  }

# Fixed random state 
randomstate = 565

In [9]:
# [j,names,n_splits,i,classifier,X_train.shape,X_test.shape,y_test.sum(),y_pred.sum(),acc_score,mcc,rfc_cv_score,aic_score,bic_score,X_train, y_train,X_test,y_test]
model_col= ['Iter','Unit','model','test_sum','pred_sum','mse','rmse','variance','mae','r2','aic_score','bic_score']

# Store the all iteration results 
model_result_df = pd.DataFrame(columns=model_col)

In [10]:
# XgBoost Model for each unit 


# CHANGE THE FILENAME EVERY TIME YOU RUN THIS CODE
# Saving model in DBFS
modelpath = "/dbfs/mnt/"
filename='sheena3_xgb_'

#fig,axes=plt.subplots(12,1,figsize=(6,80))
j=0


# Loop for each unit and cols contains columns for each unit 
for cols in units:

  # Columns of the units
  flag = cols[-1]
  flag1 = cols[-2]
#   flag2 = cols[-3]
#   dow = cols[-4]
  
  # name of the unit 
  names = cols[-1]
  print("\n UNITNAME :==================================================================",names)
  
  # getting data based on columns for each unit 
  data = dataset[cols]
  data = data[(data[flag] > 0)]

  # mapping categorical variables 
  #data[flag] = data[flag].map(fl)
  data[flag1] = data[flag1].map(fl)
#   data[flag2] = data[flag2].map(fl)
#   data[dow] = data[dow].map(day)

  
  # Selecting X and Y variables:
  X = data.iloc[:,:data.columns.get_loc(flag)].values
  y = data.iloc[:,data.columns.get_loc(flag)].values
  
  # Training and testing set 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,random_state = randomstate)
                                                      
          
  # Standard Scalling  
#   X_train = np.log(X_train)
#   X_test = np.log(X_test)
#   y_train = np.log(y_train)
#   y_test = np.log(y_test)
#   scaler = StandardScaler()
#   X_train = scaler.fit_transform(X_train)
#   X_test = scaler.fit_transform(X_test)
  

  print('X_train', X_train.shape, 'Y_train' , y_train.shape)
  
  #Regressor codes
#   xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.5, learning_rate = 0.1, max_depth = 4,alpha = 10, n_estimators = 1000,sub_sample = 0.8,random_state = randomstate)
  xg_reg = LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
  #model = Ridge()
#   visualizer = ResidualsPlot(xg_reg)
#   visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
#   visualizer.score(X_test, y_test)  # Evaluate the model on the test data
#   visualizer.show()

  
#   X2 = sm.add_constant(X)
#   est = sm.OLS(y, X2)
#   est2 = est.fit()
#   print(est2.summary())
  
                            
  xg_reg.fit(X_train,y_train)
  
   
  #mlflow.sklearn.save_model(xg_reg, modelpath + filename + names + "_" + str(j))

  # test set  prediction
  y_pred =  xg_reg.predict(X_test)
#   residual = y_test-y_pred
#   plt.scatter(y_test,residual)
#   plt.xlabel("Y Test")
#   plt.ylabel("residual")
#   display(plt.show())
  
#   y_test = np.exp(y_test)
#   y_pred = np.exp(y_pred)

  # AIC Score
  aic_score = aic.aic(y_test.tolist(),y_pred.tolist(), (len(cols)-1))
  
  # BIC Score
  bic_score = bic.bic(y_test.tolist(), y_pred.tolist(), (len(cols)-1))
  
  
  mse_test = mean_squared_error(y_test,y_pred)
  rmse_test = np.sqrt(mean_squared_error(y_test,y_pred))
  var_test = explained_variance_score(y_test, y_pred)
  mae_test = mean_absolute_error(y_test, y_pred)
  r2_test = r2_score(y_test, y_pred)
  print('MSE: ',mse_test,'    RMSE: ',rmse_test,'    Variance score: ',var_test,'    MAE: ',mae_test,'    R-square: ',r2_test,'   AIC:  ',aic_score, '  BIC: ', bic_score)

  # Row to push in a dataframe 
  row = [j,names,'xgboost',y_test.sum(),y_pred.sum(),mse_test,rmse_test,var_test,mae_test,r2_test,aic_score,bic_score]
  
  # Result data in a dataframe 
  model_result_df.loc[j] = row
  
  # Iterator updation
  j=j+1
  

UNITNAME :================================================================== HB1_mmbtu_new
X_train (276, 19) Y_train (276,)
MSE: 31649882.59025927 RMSE: 5625.822836728798 Variance score: 0.4067957284447903 MAE: 4209.089776614335 R-square: 0.3986011355187876 AIC: 1229.646904787419 BIC: 1272.094928374767

In [11]:
a = pd.DataFrame(y_test)
b = pd.DataFrame(y_pred)
c = pd.merge(a,b,left_index=True, right_index=True)
display(c)

0_x,0_y
23608.374452399996,20292.862266269425
14496.209680700002,8317.986214609504
10845.423263,19783.04165316379
7045.471792,14484.347911306977
16962.809380900002,15590.019721927369
14426.477710099998,14555.232701686997
10970.3197033,10205.701366332243
4999.0698504,13131.971487521223
8388.6844672,10458.08583498784
26996.044320999998,23927.975922402868


In [12]:
cols1 = cols[0:len(cols)-1]
cols1.insert(0, 'intercept')
a = pd.DataFrame(cols1)
b = pd.DataFrame(xg_reg.coef_)
c = pd.merge(a,b,left_index=True, right_index=True)
display(c)

0_x,0_y
intercept,-4605.400534661593
Consumer_burbank_temp_actual_lag1,6298.2563336258545
Consumer_burbank_temp_actual_lag2,-1857.7548085737012
Consumer_burbank_temp_f,1582.4198330748504
Consumer_burbank_temp_f_lead1,3564.810759616009
Consumer_burbank_temp_f_lead2,-3717.478130405743
Consumer_burbank_tempF_degreedays,-129.2450874119927
Consumer_burbank_tempF_degreedays_lead1,-4677.889863556053
Consumer_burbank_tempF_degreedays_lead2,1817.7192619305608
Consumer_long_beach_temp_actual_lag1,-7795.878265804941


In [13]:
cols1 = cols[0:len(cols)-1]
cols1.insert(0, 'intercept')
a = pd.DataFrame(cols1)
b = pd.DataFrame(est2.pvalues)
c = pd.merge(a,b,left_index=True, right_index=True)
display(c)

0_x,0_y
intercept,0.549954824511688
Consumer_burbank_temp_actual_lag1,0.5435721467430018
Consumer_burbank_temp_actual_lag2,0.11090583385180204
Consumer_burbank_temp_f,0.9937228620886769
Consumer_burbank_temp_f_lead1,0.8142847027009696
Consumer_burbank_temp_f_lead2,0.47756805882655196
Consumer_burbank_tempF_degreedays,0.4755834675376325
Consumer_burbank_tempF_degreedays_lead1,0.7173510476831209
Consumer_burbank_tempF_degreedays_lead2,0.3478994723558696
Consumer_long_beach_temp_actual_lag1,0.8850548584664537


In [14]:
display(model_result_df)

Iter,Unit,model,test_sum,pred_sum,mse,rmse,variance,mae,r2,aic_score,bic_score
0,HB1_mmbtu_new,xgboost,1147144.4843408,1054145.0172821195,4.2329344236851044E7,6506.100540020193,0.5199912885030256,3910.9622412473686,0.5170119973551213,3234.1003839654354,3294.9765110198946


In [15]:
# adsDay_val = spark.sql("select * from mu_dispatch_opt.ADS_DayLevel_22November2019_august_v2")
# # adshourLevel = spark.table("ADS_HourLevel")
# type(adsDay_val)

# # august data contains august 2019 data for validation
# validation_data = adsDay_val.toPandas()
# print("august_data_shape : ",validation_data.shape)

validation_data  = dataset[(dataset['Date'] >= '2019-07-01') & (dataset['Date'] <= '2019-07-31')]
print("august_data_shape : ",validation_data.shape)


august_data_shape : (29, 322)

In [16]:
validation_data['Year']=validation_data['Date'].apply(lambda x: x.year)
validation_data['Month']=validation_data['Date'].apply(lambda x: x.month)
validation_data['Weekofyear']=validation_data['Date'].apply(lambda x: x.weekofyear)
 
val_col=['Iter','Unit','strClassifier','test_oncount','pred_oncount','TP','TN','FP','FN','Precision','Recall','Accuracy','MCC Score','AUC Score','AIC','BIC']

unit_validation_df = pd.DataFrame(columns=val_col)


In [17]:
# Random Forest Model for each unit 
# j variable keeps the track of iteration
j=0

# Loop for each unit and cols contains columns for each unit 
for cols in units:

  # Columns of the units
  flag = cols[-1]
  flag1 = cols[-2]
  flag2 = cols[-3]
  #dow = cols[-4]
  
  # name of the unit 
  names =cols[-1]
  print("\n UNITNAME :==================================================================",names)
  
  # getting data based on columns for each unit 
  val_data = validation_data[cols]
  val_data[flag] = val_data[flag].map(fl)
  val_data[flag1] = val_data[flag1].map(fl)
  val_data[flag2] = val_data[flag2].map(fl)
  #val_data[dow] = val_data[dow].map(day)

  classifier= mlflow.sklearn.load_model(model_uri=modelpath + fn+str(j))
  
  # Selecting X and Y variables:
  val_X_test = val_data.iloc[:,:val_data.columns.get_loc(flag)].values
  val_y_test = val_data.iloc[:,val_data.columns.get_loc(flag)].values
          
  # Standard Scalling  
  scaler = StandardScaler()
  val_X_test = scaler.fit_transform(val_X_test)
#   X_test = scaler.fit_transform(X_test)

  # test set  prediction
  val_y_pred =  classifier.predict(val_X_test)

  # AIC Score
  aic_score = aic.aic(val_y_test.tolist(),val_y_pred.tolist(), (len(cols)-1))
  
  # BIC Score
  bic_score = bic.bic(val_y_test.tolist(), val_y_pred.tolist(), (len(cols)-1))
  
  # Accuracy
  acc_score = metrics.accuracy_score(val_y_test, val_y_pred)
  acc_score = acc_score * 100
  
  # AUC Score
  auc_score = roc_auc_score(val_y_test, val_y_pred)
  
  # MCC Score
  mcc = metrics.matthews_corrcoef(val_y_test, val_y_pred, sample_weight=None)
  
  # Confusion matrix 
  confusion = metrics.confusion_matrix(val_y_test, val_y_pred)

  # Various confusion matrix variables 
  TP = confusion[1, 1]
  TN = confusion[0, 0]
  FP = confusion[0, 1]
  FN = confusion[1, 0]
    
  # Precision
  precision=metrics.precision_score(val_y_test, val_y_pred)
  
  # Recall
  recall=metrics.recall_score(val_y_test, val_y_pred)
   
  # Row to push in a dataframe 
  row = [j,names,str(classifier),val_y_test.sum(),val_y_pred.sum(),TP,TN,FP,FN,precision,recall,acc_score,mcc,auc_score,aic_score,bic_score]
  
  print("Confusion matrix : ", pd.crosstab(val_y_test,val_y_pred, rownames=['Actuals']).reset_index())
  TPR = TP/(TP+FN) 
 
  FPR = FP/(FP+TN)
  print('    TPR: ',TPR,'    FPR: ',FPR)
    
  print('    TP: ',TP,'    TN: ',TN,'    FP: ',FP,'    FN: ',FN)

  print('Precision: ',precision,'    Recall: ',recall, '    Accuracy: ', acc_score,'    MCC: ',mcc , '    AUC Score: ',auc_score,'    AIC Score: ', aic_score, '    BIC Score: ', bic_score)
  # Result data in a dataframe 
  unit_validation_df.loc[j] = row
  
  # Iterator updation
  j=j+1
  

UNITNAME :================================================================== Flag_AL1
Confusion matrix : col_0 Actuals 0 1
0 0 18 3
1 1 5 3
 TPR: 0.375 FPR: 0.14285714285714285
 TP: 3 TN: 18 FP: 3 FN: 5
Precision: 0.5 Recall: 0.375 Accuracy: 72.41379310344827 MCC: 0.2561358820277077 AUC Score: 0.6160714285714286 AIC Score: 40.65222563910749 BIC Score: 93.97676300857998

 UNITNAME :================================================================== Flag_AL2
Confusion matrix : col_0 Actuals 0 1
0 0 12 2
1 1 8 7
 TPR: 0.4666666666666667 FPR: 0.14285714285714285
 TP: 7 TN: 12 FP: 2 FN: 8
Precision: 0.7777777777777778 Recall: 0.4666666666666667 Accuracy: 65.51724137931035 MCC: 0.349754259914075 AUC Score: 0.661904761904762 AIC Score: 35.12338862721958 BIC Score: 80.24415101677323

 UNITNAME :================================================================== Flag_AL4
Confusion matrix : col_0 Actuals 0 1
0 0 13 1
1 1 6 9
 TPR: 0.6 FPR: 0.07142857142857142
 TP: 9 TN: 13 FP: 1 FN: 6
Precision: 0.9 Recall: 0.6 Accuracy: 75.86206896551724 MCC: 0.5556950534971523 AUC Score: 0.7642857142857143 AIC Score: 58.77981525299634 BIC Score: 127.14460675232004

 UNITNAME :================================================================== Flag_AL4
Confusion matrix : col_0 Actuals 0 1
0 0 13 1
1 1 2 13
 TPR: 0.8666666666666667 FPR: 0.07142857142857142
 TP: 13 TN: 13 FP: 1 FN: 2
Precision: 0.9285714285714286 Recall: 0.8666666666666667 Accuracy: 89.65517241379311 MCC: 0.7952380952380952 AUC Score: 0.8976190476190476 AIC Score: 10.208177301767435 BIC Score: 62.165418841253455

 UNITNAME :================================================================== Flag_AL5
Confusion matrix : col_0 Actuals 0 1
0 0 27 1
1 1 1 0
 TPR: 0.0 FPR: 0.03571428571428571
 TP: 0 TN: 27 FP: 1 FN: 1
Precision: 0.0 Recall: 0.0 Accuracy: 93.10344827586206 MCC: -0.03571428571428571 AUC Score: 0.48214285714285715 AIC Score: 20.449689166630662 BIC Score: 87.4471848359679

 UNITNAME :================================================================== Flag_AL6
Confusion matrix : col_0 Actuals 0 1
0 0 23 0
1 1 5 1
 TPR: 0.16666666666666666 FPR: 0.0
 TP: 1 TN: 23 FP: 0 FN: 5
Precision: 1.0 Recall: 0.16666666666666666 Accuracy: 82.75862068965517 MCC: 0.3700064349504774 AUC Score: 0.5833333333333334 AIC Score: 47.022120390981165 BIC Score: 114.01961606031841

 UNITNAME :================================================================== Flag_HB1
Confusion matrix : col_0 Actuals 0 1
0 0 7 2
1 1 7 13
 TPR: 0.65 FPR: 0.2222222222222222
 TP: 13 TN: 7 FP: 2 FN: 7
Precision: 0.8666666666666667 Recall: 0.65 Accuracy: 68.96551724137932 MCC: 0.39604526490270253 AUC Score: 0.7138888888888888 AIC Score: 58.06793367314262 BIC Score: 120.96354185252042

 UNITNAME :================================================================== Flag_HB2
Confusion matrix : col_0 Actuals 0 1
0 0 15 1
1 1 3 10
 TPR: 0.7692307692307693 FPR: 0.0625
 TP: 10 TN: 15 FP: 1 FN: 3
Precision: 0.9090909090909091 Recall: 0.7692307692307693 Accuracy: 86.20689655172413 MCC: 0.7243576850866262 AUC Score: 0.8533653846153846 AIC Score: 18.55095740286908 BIC Score: 70.50819894235511

 UNITNAME :================================================================== Flag_RB5
Confusion matrix : col_0 Actuals 0 1
0 0 19 0
1 1 5 5
 TPR: 0.5 FPR: 0.0
 TP: 5 TN: 19 FP: 0 FN: 5
Precision: 1.0 Recall: 0.5 Accuracy: 82.75862068965517 MCC: 0.6291528696058959 AUC Score: 0.75 AIC Score: 45.022120390981165 BIC Score: 110.65232023033192

 UNITNAME :================================================================== Flag_RB6
Confusion matrix : col_0 Actuals 0 1
0 0 19 0
1 1 7 3
 TPR: 0.3 FPR: 0.0
 TP: 3 TN: 19 FP: 0 FN: 7
Precision: 1.0 Recall: 0.3 Accuracy: 75.86206896551724 MCC: 0.468220855185637 AUC Score: 0.65 AIC Score: 32.77981525299634 BIC Score: 83.36976096249589

 UNITNAME :================================================================== Flag_RB7
Confusion matrix : col_0 Actuals 0 1
0 0 26 1
1 1 1 1
 TPR: 0.5 FPR: 0.037037037037037035
 TP: 1 TN: 26

In [18]:
display(unit_validation_df)

Iter,Unit,strClassifier,test_oncount,pred_oncount,TP,TN,FP,FN,Precision,Recall,Accuracy,MCC Score,AUC Score,AIC,BIC
0,Flag_AL1,"XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan, n_estimators=1000, n_jobs=1, nthread=4, objective='binary:logistic', random_state=565, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=None, subsample=1, verbosity=1)",8,6,3,18,3,5,0.5,0.375,72.41379310344827,0.2561358820277077,0.6160714285714286,40.65222563910749,93.97676300857998
1,Flag_AL2,"XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan, n_estimators=1000, n_jobs=1, nthread=4, objective='binary:logistic', random_state=565, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=None, subsample=1, verbosity=1)",15,9,7,12,2,8,0.7777777777777778,0.4666666666666667,65.51724137931035,0.349754259914075,0.661904761904762,35.12338862721958,80.24415101677323
2,Flag_AL4,"XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan, n_estimators=1000, n_jobs=1, nthread=4, objective='binary:logistic', random_state=565, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=None, subsample=1, verbosity=1)",15,10,9,13,1,6,0.9,0.6,75.86206896551724,0.5556950534971523,0.7642857142857143,58.77981525299634,127.14460675232004
3,Flag_AL4,"XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan, n_estimators=1000, n_jobs=1, nthread=4, objective='binary:logistic', random_state=565, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=None, subsample=1, verbosity=1)",15,14,13,13,1,2,0.9285714285714286,0.8666666666666667,89.65517241379311,0.7952380952380952,0.8976190476190476,10.208177301767435,62.165418841253455
4,Flag_AL5,"XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan, n_estimators=1000, n_jobs=1, nthread=4, objective='binary:logistic', random_state=565, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=None, subsample=1, verbosity=1)",1,1,0,27,1,1,0.0,0.0,93.10344827586206,-0.03571428571428571,0.48214285714285715,20.449689166630662,87.4471848359679
5,Flag_AL6,"XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan, n_estimators=1000, n_jobs=1, nthread=4, objective='binary:logistic', random_state=565, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=None, subsample=1, verbosity=1)",6,1,1,23,0,5,1.0,0.16666666666666666,82.75862068965517,0.3700064349504774,0.5833333333333334,47.022120390981165,114.01961606031841
6,Flag_HB1,"XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan, n_estimators=1000, n_jobs=1, nthread=4, objective='binary:logistic', random_state=565, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=None, subsample=1, verbosity=1)",20,15,13,7,2,7,0.8666666666666667,0.65,68.96551724137932,0.39604526490270253,0.7138888888888888,58.06793367314262,120.96354185252042
7,Flag_HB2,"XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan

In [19]:
# Random Forest Model for each unit 
# j variable keeps the track of iteration
j=0

# Loop for each unit and cols contains columns for each unit 
for cols in units:

  # Columns of the units
  flag = cols[-1]
  flag1 = cols[-2]
  flag2 = cols[-3]
  dow = cols[-4]
  
  # name of the unit 
  names =cols[-1]
  print("\n UNITNAME :==================================================================",names)
  
  # getting data based on columns for each unit 
  data = dataset[cols]

  # mapping categorical variables 
  data[flag] = data[flag].map(fl)
  data[flag1] = data[flag1].map(fl)
  data[flag2] = data[flag2].map(fl)
  data[dow] = data[dow].map(day)

  
  # Selecting X and Y variables:
  X = data.iloc[:,:data.columns.get_loc(flag)]
  y = data.iloc[:,data.columns.get_loc(flag)]
  
  # Training and testing set 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = randomstate,stratify=y)
          
  # Standard Scalling  
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.fit_transform(X_test)

  if j in [0,4,5,8,10,11]:
    sm = SMOTE(random_state = randomstate )
    X_train, y_train= sm.fit_sample(X_train, y_train.ravel())
 
  # Classifier for the list
  classifier =  RandomForestClassifier(n_estimators = 1000, criterion = 'gini', random_state = randomstate)
  classifier.fit(X_train, y_train)
       
  # Saving model in DBFS
  modelpath = "/mnt/dop_2"
  fn='RF_model_startify'
  mlflow.sklearn.save_model(classifier, modelpath + fn + str(j))

  # test set  prediction
  y_pred =  classifier.predict(X_test)

  # AIC Score
  aic_score = aic.aic(y_test.tolist(),y_pred.tolist(), (len(cols)-1))
  
  # BIC Score
  bic_score = bic.bic(y_test.tolist(), y_pred.tolist(), (len(cols)-1))
  
  # Accuracy
  acc_score = metrics.accuracy_score(y_test, y_pred)
  acc_score = acc_score * 100
  
  # AUC Score
  auc_score = roc_auc_score(y_test, y_pred)
  
  # MCC Score
  mcc = metrics.matthews_corrcoef(y_test, y_pred, sample_weight=None)
  
  # Confusion matrix 
  confusion = metrics.confusion_matrix(y_test, y_pred)

  # Various confusion matrix variables 
  TP = confusion[1, 1]
  TN = confusion[0, 0]
  FP = confusion[0, 1]
  FN = confusion[1, 0]
    
  # Precision
  precision=metrics.precision_score(y_test, y_pred)
  
  # Recall
  recall=metrics.recall_score(y_test, y_pred)
   
  # Row to push in a dataframe 
  row = [j,names,str(classifier),X_train.shape,X_test.shape,y_test.sum(),y_pred.sum(),TP,TN,FP,FN,precision,recall,acc_score,mcc,auc_score,aic_score,bic_score]
  
  print("Confusion matrix : ", pd.crosstab(y_test,y_pred, rownames=['Actuals']).reset_index())
  
  print('Precision: ',precision,'    Recall: ',recall, '    Accuracy: ', acc_score,'    MCC: ',mcc , '    AUC Score: ',auc_score,'    AIC Score: ', aic_score, '    BIC Score: ', bic_score)
  # Result data in a dataframe 
  model_result_df.loc[j] = row
  
  # Iterator updation
  j=j+1
  

UNITNAME :================================================================== Flag_AL1
Confusion matrix : col_0 Actuals 0 1
0 0 151 45
1 1 4 31
Precision: 0.40789473684210525 Recall: 0.8857142857142857 Accuracy: 78.78787878787878 MCC: 0.500696216713285 AUC Score: 0.8280612244897959 AIC Score: -288.1880022669796 BIC Score: -167.70338239871683

 UNITNAME :================================================================== Flag_AL2
Confusion matrix : col_0 Actuals 0 1
0 0 147 13
1 1 23 48
Precision: 0.7868852459016393 Recall: 0.676056338028169 Accuracy: 84.4155844155844 MCC: 0.6225516307012215 AUC Score: 0.7974031690140845 AIC Score: -373.4056163471729 BIC Score: -277.0179204525627

 UNITNAME :================================================================== Flag_AL3
Confusion matrix : col_0 Actuals 0 1
0 0 123 18
1 1 12 78
Precision: 0.8125 Recall: 0.8666666666666667 Accuracy: 87.01298701298701 MCC: 0.7312685847927606 AUC Score: 0.8695035460992908 AIC Score: -395.52189596657644 BIC Score: -264.7100229667483

 UNITNAME :================================================================== Flag_AL4
Confusion matrix : col_0 Actuals 0 1
0 0 127 14
1 1 15 75
Precision: 0.8426966292134831 Recall: 0.8333333333333334 Accuracy: 87.44588744588745 MCC: 0.735551139589004 AUC Score: 0.8670212765957447 AIC Score: -399.35315440365883 BIC Score: -261.6564459827871

 UNITNAME :================================================================== Flag_AL5
Confusion matrix : col_0 Actuals 0 1
0 0 211 2
1 1 5 13
Precision: 0.8666666666666667 Recall: 0.7222222222222222 Accuracy: 96.96969696969697 MCC: 0.775427273994029 AUC Score: 0.85641627543036 AIC Score: -733.693246698757 BIC Score: -606.3237914094507

 UNITNAME :================================================================== Flag_AL6
Confusion matrix : col_0 Actuals 0 1
0 0 204 4
1 1 5 18
Precision: 0.8181818181818182 Recall: 0.782608695652174 Accuracy: 96.1038961038961 MCC: 0.778665082035041 AUC Score: 0.8816889632107023 AIC Score: -667.6396137658676 BIC Score: -526.5004876344741

 UNITNAME :================================================================== Flag_HB1
Confusion matrix : col_0 Actuals 0 1
0 0 116 16
1 1 8 91
Precision: 0.8504672897196262 Recall: 0.9191919191919192 Accuracy: 89.6103896103896 MCC: 0.7919426421321772 AUC Score: 0.8989898989898989 AIC Score: -431.0680563201589 BIC Score: -272.71684163615646

 UNITNAME :================================================================== Flag_HB2
Confusion matrix : col_0 Actuals 0 1
0 0 155 5
1 1 14 57
Precision: 0.9193548387096774 Recall: 0.8028169014084507 Accuracy: 91.77489177489177 MCC: 0.8033847020363475 AUC Score: 0.8857834507042254 AIC Score: -493.0330869430866 BIC Score: -348.4515431011713

 UNITNAME :================================================================== Flag_RB5
Confusion matrix : col_0 Actuals 0 1
0 0 182 13
1 1 8 28
Precision: 0.6829268292682927 Recall: 0.7777777777777778 Accuracy: 90.9090909090909 MCC: 0.6750520986566351 AUC Score: 0.8555555555555555 AIC Score: -479.9138080164237 BIC Score: -352.5443527271173

 UNITNAME :================================================================== Flag_RB6
Confusion matrix : col_0 Actuals 0 1
0 0 160 13
1 1 11 47
Precision: 0.7833333333333333 Recall: 0.8103448275862069 Accuracy: 89.6103896103896 MCC: 0.7270579954291462 AUC Score: 0.8676001594578433 AIC Score: -467.0680563201589 BIC Score: -370.6803604255487

 UNITNAME :================================================================== Flag_RB7
Confusion matrix : col_0 Actuals 0 1
0 0 205 4
1 1 2 20
Precision: 0.8333333333333334 Recall: 0.9090909090909091 Accuracy: 97.40259740259741 MCC: 0.8561706738324665 AUC Score: 0.944976076555024 AIC Score: -765.3020537388535 BIC Score: -631.0477630285036

 UNITNAME :================================================================== Flag_RB8
Confusion matrix : col_0 Actuals 0 1
0 0 208 6
1 1 1 16
Precision: 0.7272727272727273 Recall: 0.9411764705882353 Accuracy: 96.96969696969697 MCC: 0.812

In [20]:

# j variable keeps the track of iteration
j=0

# Loop for each unit and cols contains columns for each unit 
for cols in units:

  # Columns of the units
  flag = cols[-1]
  flag1 = cols[-2]
  flag2 = cols[-3]
  dow = cols[-4]
  
  # name of the unit 
  names =cols[-1]
  print("\n UNITNAME :==================================================================",names)
  
  # getting data based on columns for each unit 
  val_data = validation_data[cols]
  val_data[flag] = val_data[flag].map(fl)
  val_data[flag1] = val_data[flag1].map(fl)
  val_data[flag2] = val_data[flag2].map(fl)
  val_data[dow] = val_data[dow].map(day)

  modelpath = "/mnt/dop_2"
  classifier= mlflow.sklearn.load_model(model_uri=modelpath + fn+str(j))
  
  # Selecting X and Y variables:
  val_X_test = val_data.iloc[:,:val_data.columns.get_loc(flag)].values
  val_y_test = val_data.iloc[:,val_data.columns.get_loc(flag)].values
          
  # Standard Scalling  
  scaler = StandardScaler()
  val_X_test = scaler.fit_transform(val_X_test)
#   X_test = scaler.fit_transform(X_test)

  # test set  prediction
  val_y_pred =  classifier.predict(val_X_test)

  # AIC Score
  aic_score = aic.aic(val_y_test.tolist(),val_y_pred.tolist(), (len(cols)-1))
  
  # BIC Score
  bic_score = bic.bic(val_y_test.tolist(), val_y_pred.tolist(), (len(cols)-1))
  
  # Accuracy
  acc_score = metrics.accuracy_score(val_y_test, val_y_pred)
  acc_score = acc_score * 100
  
  # AUC Score
  auc_score = roc_auc_score(val_y_test, val_y_pred)
  
  # MCC Score
  mcc = metrics.matthews_corrcoef(val_y_test, val_y_pred, sample_weight=None)
  
  # Confusion matrix 
  confusion = metrics.confusion_matrix(val_y_test, val_y_pred)

  # Various confusion matrix variables 
  TP = confusion[1, 1]
  TN = confusion[0, 0]
  FP = confusion[0, 1]
  FN = confusion[1, 0]
    
  # Precision
  precision=metrics.precision_score(val_y_test, val_y_pred)
  
  # Recall
  recall=metrics.recall_score(val_y_test, val_y_pred)
   
  # Row to push in a dataframe 
  row = [j,names,str(classifier),X_train.shape,X_test.shape,val_y_test.sum(),val_y_pred.sum(),TP,TN,FP,FN,precision,recall,acc_score,mcc,auc_score,aic_score,bic_score]
  
  print("Confusion matrix : ", pd.crosstab(val_y_test,val_y_pred, rownames=['Actuals']).reset_index())
  
  print('Precision: ',precision,'    Recall: ',recall, '    Accuracy: ', acc_score,'    MCC: ',mcc , '    AUC Score: ',auc_score,'    AIC Score: ', aic_score, '    BIC Score: ', bic_score)
  # Result data in a dataframe 
  unit_validation_df.loc[j] = row
  
  # Iterator updation
  j=j+1
  

UNITNAME :================================================================== Flag_AL1
Confusion matrix : col_0 Actuals 0 1
0 0 15 3
1 1 3 6
Precision: 0.6666666666666666 Recall: 0.6666666666666666 Accuracy: 77.77777777777779 MCC: 0.5 AUC Score: 0.75 AIC Score: 29.3899102870406 BIC Score: 74.74420059719213

 UNITNAME :================================================================== Flag_AL2
Confusion matrix : col_0 Actuals 0 1
0 0 6 2
1 1 7 12
Precision: 0.8571428571428571 Recall: 0.631578947368421 Accuracy: 66.66666666666666 MCC: 0.34871490092681434 AUC Score: 0.6907894736842105 AIC Score: 26.337468205961034 BIC Score: 62.620900454082246

 UNITNAME :================================================================== Flag_AL3
Confusion matrix : col_0 Actuals 0 1
0 0 0 2
1 1 2 23
Precision: 0.92 Recall: 0.92 Accuracy: 85.18518518518519 MCC: -0.08 AUC Score: 0.46 AIC Score: 24.44235236812016 BIC Score: 73.68415327628466

 UNITNAME :================================================================== Flag_AL4
Confusion matrix : col_0 Actuals 0 1
0 0 0 1
1 1 2 24
Precision: 0.96 Recall: 0.9230769230769231 Accuracy: 88.88888888888889 MCC: -0.05547001962252291 AUC Score: 0.46153846153846156 AIC Score: 20.67493641192207 BIC Score: 72.50841105209523

 UNITNAME :================================================================== Flag_AL5
Confusion matrix : col_0 Actuals 0 1
0 0 20 0
1 1 6 1
Precision: 1.0 Recall: 0.14285714285714285 Accuracy: 77.77777777777779 MCC: 0.33149677206589795 AUC Score: 0.5714285714285714 AIC Score: 33.3899102870406 BIC Score: 81.33587432920078

 UNITNAME :================================================================== Flag_AL6
Confusion matrix : col_0 Actuals 0 1
0 0 12 0
1 1 13 2
Precision: 1.0 Recall: 0.13333333333333333 Accuracy: 51.85185185185185 MCC: 0.2529822128134704 AUC Score: 0.5666666666666667 AIC Score: 62.26603726934461 BIC Score: 115.3953487755221

 UNITNAME :================================================================== Flag_HB1
Confusion matrix : col_0 Actuals 0 1
0 0 1 1
1 1 1 24
Precision: 0.96 Recall: 0.96 Accuracy: 92.5925925925926 MCC: 0.46 AUC Score: 0.73 AIC Score: 21.727378493001638 BIC Score: 81.33587432920079

 UNITNAME :================================================================== Flag_HB2
Confusion matrix : col_0 Actuals 0 1
0 0 6 1
1 1 7 13
Precision: 0.9285714285714286 Recall: 0.65 Accuracy: 70.37037037037037 MCC: 0.44479370979142197 AUC Score: 0.7535714285714286 AIC Score: 51.157326243238685 BIC Score: 105.5824746154205

 UNITNAME :================================================================== Flag_RB5
Confusion matrix : col_0 Actuals 0 1
0 0 7 4
1 1 4 12
Precision: 0.75 Recall: 0.75 Accuracy: 70.37037037037037 MCC: 0.38636363636363635 AUC Score: 0.6931818181818181 AIC Score: 41.157326243238685 BIC Score: 89.10329028539886

 UNITNAME :================================================================== Flag_RB6
Confusion matrix : col_0 Actuals 0 1
0 0 4 3
1 1 8 12
Precision: 0.8 Recall: 0.6 Accuracy: 59.25925925925925 MCC: 0.15118578920369088 AUC Score: 0.5857142857142856 AIC Score: 31.75557698343912 BIC Score: 68.03900923156033

 UNITNAME :================================================================== Flag_RB7
Confusion matrix : col_0 Actuals 0
0 0 22
1 1 5
Precision: 0.0 Recall: 0.0 Accuracy: 81.48148148148148 MCC: 0.0 AUC Score: 0.5 AIC Score: 32.46722825360382 BIC Score: 83.00486602777266

 UNITNAME :================================================================== Flag_RB8
Confusion matrix : col_0 Actuals 0 1
0 0 20 0
1 1 6 1
Precision: 1.0 Recall: 0.14285714285714285 Accuracy: 77.77777777777779 MCC: 0.33149677206589795 AUC Score: 0.5714285714285714 AIC Score: 37.3899102870406 BIC Score: 87.92754806120945

In [21]:
display(unit_validation_df)

Iter,Unit,strClassifier,Train shape,Test shape,test_oncount,pred_oncount,TP,TN,FP,FN,Precision,Recall,Accuracy,MCC Score,AUC Score,AIC,BIC
0,Flag_AL1,"RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None, oob_score=False, random_state=565, verbose=0, warm_start=False)","List(1704, 39)","List(231, 39)",9,9,6,15,3,3,0.6666666666666666,0.6666666666666666,77.77777777777779,0.5,0.75,29.3899102870406,74.74420059719213
1,Flag_AL2,"RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None, oob_score=False, random_state=565, verbose=0, warm_start=False)","List(1704, 39)","List(231, 39)",19,14,12,6,2,7,0.8571428571428571,0.631578947368421,66.66666666666666,0.34871490092681434,0.6907894736842105,26.337468205961034,62.620900454082246
2,Flag_AL3,"RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None, oob_score=False, random_state=565, verbose=0, warm_start=False)","List(1704, 39)","List(231, 39)",25,25,23,0,2,2,0.92,0.92,85.18518518518519,-0.08,0.46,24.44235236812016,73.68415327628466
3,Flag_AL4,"RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None, oob_score=False, random_state=565, verbose=0, warm_start=False)","List(1704, 39)","List(231, 39)",26,25,24,0,1,2,0.96,0.9230769230769231,88.88888888888889,-0.05547001962252291,0.46153846153846156,20.67493641192207,72.50841105209523
4,Flag_AL5,"RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None, oob_score=False, random_state=565, verbose=0, warm_start=False)","List(1704, 39)","List(231, 39)",7,1,1,20,0,6,1.0,0.14285714285714285,77.77777777777779,0.33149677206589795,0.5714285714285714,33.3899102870406,81.33587432920078
5,Flag_AL6,"RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None, oob_score=False, random_state=565, verbose=0, warm_start=False)","List(1704, 39)","List(231, 39)",15,2,2,12,0,13,1.0,0.13333333333333333,51.85185185185185,0.2529822128134704,0.5666666666666667,62.26603726934461,115.3953487755221
6,Flag_HB1,"RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None, oob_score=False, random_state=565, verbose=0, warm_start=False)","List(1704, 39)","List(231, 39)",25,25,24,1,1,1,0.96,0.96,92.5925925925926,0.46,0.73,21.727378493001638,81.33587432920079
7,Flag_HB2,"RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split